<a href="https://colab.research.google.com/github/YuevUwU/phira/blob/build-sh/build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Set Repository URL** (~0.5s)

In [ ]:
%env REPO_URL=https://github.com/TeamFlos/phira
%env BRANCH=tags/v0.6.7

# **Before building**  (~47s)

## **Setup Rust and Common Dependencies** (~43s)

In [ ]:
!apt-get install -y curl zip unzip gcc make
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --profile=minimal --default-toolchain nightly

## **Clone Phira repository** (~2s)

In [ ]:
%cd /content/
!git clone $REPO_URL
%cd /content/phira
!git checkout $BRANCH

## **Download prebuilt FFmpeg static library** (~2s)

In [ ]:
%cd /content/phira/prpr-avc
!curl -LO https://github.com/TeamFlos/phira/files/14319201/static-lib.zip
!unzip static-lib.zip -d static-lib
!rm static-lib.zip

# **Windows x86_64 (unoptimized, debug)** (~30m)

## **Install Dependencies** (~25s)

In [ ]:
!apt-get install -y libfindbin-libs-perl gcc-mingw-w64

## **Install Toolchain** (~6s)

In [ ]:
%cd /content/phira
!/root/.cargo/bin/rustup +nightly target add x86_64-pc-windows-gnu

## **Update ahash ([PR#438](https://github.com/TeamFlos/phira/pull/438))** (~5s)

In [ ]:
%cd /content/phira
!sed -i 's/ahash = "=0\.8\.6"/ahash = "=0.8.7"/' prpr/Cargo.toml

## **Build for Windows x86_64** (~29m)

In [ ]:
%cd /content/phira
!/root/.cargo/bin/cargo +nightly build --target=x86_64-pc-windows-gnu --bin phira-main

## **Packaging for Windows x86_64** (~28s)

In [ ]:
!curl -s https://api.github.com/repos/TeamFlos/phira/releases/latest \
  | grep -o '"browser_download_url": *"[^"]*"' \
  | grep "windows" \
  | grep ".zip\"" \
  | sed 's/"browser_download_url": "//' \
  | sed 's/"$//' \
  | xargs -n 1 curl -o latest-windows.zip -L

!mkdir -p out/x86_64-pc-windows-gnu
!unzip latest-windows.zip -d out/x86_64-pc-windows-gnu
!cp -rf assets out/x86_64-pc-windows-gnu/
!cp -f target/x86_64-pc-windows-gnu/debug/phira-main.exe out/x86_64-pc-windows-gnu/phira-main.exe

%cd /content/phira/out/x86_64-pc-windows-gnu

zip_name="Phira-windows-$(date +%s%N).zip"
!zip -rq "/content/$zip_name" .

# **Linux x86_64 (unoptimized, debug)** (First run: ~23m)

## **Install Dependencies** (~33s)

In [ ]:
!apt-get install -y perl pkg-config libgtk-3-dev libasound2-dev

## **Install Toolchain** (~0.3s)

In [ ]:
%cd /content/phira
!/root/.cargo/bin/rustup +nightly target add x86_64-unknown-linux-gnu

## **Update ahash ([PR#438](https://github.com/TeamFlos/phira/pull/438))** (~4s)

In [ ]:
%cd /content/phira
!sed -i 's/ahash = "=0\.8\.6"/ahash = "=0.8.7"/' prpr/Cargo.toml

## **Build for Linux x86_64** (First run: ~22m)

In [ ]:
%cd /content/phira
!/root/.cargo/bin/cargo +nightly build --target=x86_64-unknown-linux-gnu --bin phira-main

## **Packaging for Linux x86_64** (~22s)

In [ ]:
%cd /content/phira

!curl -s https://api.github.com/repos/TeamFlos/phira/releases/latest \
  | grep -o '"browser_download_url": *"[^"]*"' \
  | grep "linux" \
  | grep ".zip\"" \
  | sed 's/"browser_download_url": "//' \
  | sed 's/"$//' \
  | xargs -n 1 curl -o latest-linux.zip -L

!mkdir -p out/x86_64-unknown-linux-gnu
!unzip latest-linux.zip -d out/x86_64-unknown-linux-gnu
!cp -rf assets out/x86_64-unknown-linux-gnu/
!cp -f target/x86_64-unknown-linux-gnu/debug/phira-main out/x86_64-unknown-linux-gnu/phira-main

%cd /content/phira/out/x86_64-unknown-linux-gnu

zip_name="Phira-linux-$(date +%s%N).zip"
!zip -rq "/content/$zip_name" .